In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

In [3]:
pd.set_option('max_columns', None)

In [4]:
combined_data_file = "../../data/Data-Jul16/combined_jul16.csv"
data_2020_file = "../../data/Data-Jul16/F20_jul05.csv"
data_historical = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")

In [5]:
data_2020 = pd.read_csv(data_2020_file, encoding = "ISO-8859-1")
data_2016 = data_historical.loc[data_historical['YEAR'] == 2016]
data_2012 = data_historical.loc[data_historical['YEAR'] == 2012]
data_2008 = data_historical.loc[data_historical['YEAR'] == 2008]

year_df_dict = {2020:data_2020, 2016:data_2016, 2012:data_2012, 2008:data_2008}

In [6]:
data_historical['COUNTY_TOTALVOTES'] = data_historical['COUNTY_TOTALVOTES'].astype(np.int64)

### PrepData###

In [7]:
data_historical['REP_VOTES%'] = data_historical['REP_VOTES'] / data_historical['COUNTY_TOTALVOTES']
data_historical['DEM_VOTES%'] = data_historical['DEM_VOTES'] / data_historical['COUNTY_TOTALVOTES']
data_2020['REP_VOTES%'] = data_2020['REP_VOTES'] / data_2020['COUNTY_TOTALVOTES']
data_2020['DEM_VOTES%'] = data_2020['DEM_VOTES']/ data_2020['COUNTY_TOTALVOTES']

In [8]:
data_historical.drop(['REP_VOTES', 'DEM_VOTES'], axis=1, inplace=True)
data_2020.drop(['REP_VOTES', 'DEM_VOTES'], axis=1, inplace=True)

In [9]:
data_historical.drop(['WINNING_CANDIDATE', 'WINNING_PARTY_BINARY', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'AVG_WAGE_SALARY', 'WINNING_PARTY', 'COUNTY', 'STATE', 'REP_VOTES%', 'DEM_VOTES%', 'MARGIN_VICTORY'], axis=1, inplace=True)

In [10]:
def train_test_split_by_year(X, y, year, cols=None):
    
    if year != None:
        year_filter =  X['YEAR'] < year
        X = X[year_filter]
        y = y[year_filter]
        
    X = X.drop('YEAR', axis=1)
    y = np.delete(y, 1, axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    if (cols == None):
        return X_train, X_test, y_train, y_test
    else:
        return X_train[cols], X_test[cols], y_train, y_test

# Step 1 #

### Build/Train Model#1: Linear regression to preidct total votes by county###


In [11]:
year = 2020
target_df = year_df_dict.get(year)

In [12]:
X = data_historical.drop(['COUNTY_TOTALVOTES'], axis=1)
y = data_historical[['COUNTY_TOTALVOTES', 'YEAR']].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split_by_year(X, y, year)

In [14]:
linear_model = LinearRegression()

In [15]:
linear_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [16]:
y_pred = linear_model.predict(X_test)

In [17]:
print("Accuracy score={:.2f}".format(linear_model.score(X_test, y_test) * 100))

Accuracy score=98.76


### Generate Total Votes Predictions by County###

In [18]:
target_df.drop(['COUNTY_TOTALVOTES'], inplace=True, axis=1, errors='ignore')

In [19]:
target_df.drop(['ID', 'YEAR', 'Unnamed: 0', 'Unnamed: 0.1', 'WINNING_CANDIDATE', 'WINNING_PARTY_BINARY', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE', 'REP_VOTES%', 'DEM_VOTES%', 'REP_VOTES', 'DEM_VOTES', 'COUNTY_TOTALVOTES', 'MARGIN_VICTORY', 'AVG_WAGE_SALARY'], axis=1, inplace=True, errors='ignore')

In [20]:
y_pred_target = linear_model.predict(target_df)

In [21]:
#merge predictions with the target dataset. 
target_df['COUNTY_TOTALVOTES'] = y_pred_target

In [22]:
target_df['COUNTY_TOTALVOTES'] = target_df['COUNTY_TOTALVOTES'].astype(np.int64)

In [23]:
import datetime

x = datetime.datetime.now()
day = x.day
month = x.month

date_str = str(month) + str(day)

In [24]:
#save file (will be used by next step in the pipeline)
filename_step1 = '../../data/F20_step1_output_' + date_str + '.csv'
target_df.to_csv(filename_step1)

# Step 2 #

In [25]:
# start clean and reimport everything again. 
data = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")

In [26]:
data_2020 = pd.read_csv(data_2020_file, encoding = "ISO-8859-1")
data_2016 = data.loc[data['YEAR'] == 2016]
data_2012 = data.loc[data['YEAR'] == 2012]
data_2008 = data.loc[data['YEAR'] == 2008]

### Build/Train Model#2: Random Forest Classifier to predict County Winner###

In [27]:
data.drop(['REP_VOTES', 'DEM_VOTES', 'MARGIN_VICTORY', 'WINNING_CANDIDATE', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)
data.drop(['AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE', 'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE' , 'TOT_FEMALE', 'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19', 'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39', 'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59', 'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60', 'TOT_MALE_Above60', 'TOT_FEMALE_Above60'] , axis=1, inplace=True)


In [28]:
significant_cols = [
 'STATE_FIPS',
 'COUNTY_FIPS',
 'COUNTY_TOTALVOTES',
 'HOUSE_WINNING_BINARY',
 'SENATE_WINNING_BINARY',
 'UNEMPLOYMENT_RATE',
 'BA_FEMALE%',
 'BA_MALE%',
 'H_FEMALE%',
 'IA_FEMALE%',
 'WA_FEMALE%',
 'WA_MALE%',
 'TOT_FEMALE%',
 'TOT_MALE%',
 'TOT_POP_LESS19%',
 'TOT_POP_40to59%',
 'TOT_POP_Above60%']

In [29]:
X = data.drop('WINNING_PARTY_BINARY', axis=1)
y = data[['WINNING_PARTY_BINARY', 'YEAR']].values

X_train, X_test, y_train, y_test = train_test_split_by_year(X, y, year, significant_cols)

In [30]:
rfc4 = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

rfc_pred4= rfc4.predict(X_test)

print("Experiment#rfc4: {:.3f}%".format(accuracy_score(y_test, rfc_pred4) * 100))

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


Experiment#rfc4: 89.153%


In [31]:
print(classification_report(y_test, rfc_pred4))

              precision    recall  f1-score   support

           0       0.81      0.64      0.72      1006
           1       0.91      0.96      0.93      3714

   micro avg       0.89      0.89      0.89      4720
   macro avg       0.86      0.80      0.82      4720
weighted avg       0.89      0.89      0.89      4720



### Predict County winner ###

In [32]:
target_year_data = pd.read_csv(filename_step1, encoding = "ISO-8859-1")

In [33]:
## use with significant model only..
## for now drop AVG_WAGE_SALARY till data is fixed. 
data_significant = target_year_data[[
 'STATE_FIPS',
 'COUNTY_FIPS',
 'COUNTY_TOTALVOTES',
 'HOUSE_WINNING_BINARY',
 'SENATE_WINNING_BINARY',
 'UNEMPLOYMENT_RATE',
 'BA_FEMALE%',
 'BA_MALE%',
 'H_FEMALE%',
 'IA_FEMALE%',
 'WA_FEMALE%',
 'WA_MALE%',
 'TOT_FEMALE%',
 'TOT_MALE%',
 'TOT_POP_LESS19%',
 'TOT_POP_40to59%',
 'TOT_POP_Above60%']]

In [34]:
y_pred = rfc4.predict(data_significant)

In [35]:
target_year_data['WINNING_PARTY_BINARY'] = y_pred

In [36]:
#save to csv
filename_step2 = '../../data/F20_step2_output_' + date_str + '.csv'
target_year_data.to_csv(filename_step2)

# Step 3

### Build/Train Model#3: Linear Regression to predict R/D Votes for every county###

In [37]:
votes_historcail_df = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")
votes_df = pd.read_csv(filename_step2)

In [38]:
votes_historcail_df_copy = votes_historcail_df[['YEAR','STATE_FIPS', 'COUNTY_FIPS', 'COUNTY_TOTALVOTES', 'WINNING_PARTY_BINARY', 'REP_VOTES']]

In [39]:
votes_historcail_df_copy['REP_VOTES%'] = votes_historcail_df_copy['REP_VOTES'] / votes_historcail_df_copy['COUNTY_TOTALVOTES']
votes_historcail_df_copy.drop('REP_VOTES', axis=1, inplace=True)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [40]:
X = votes_historcail_df_copy.drop(['REP_VOTES%'], axis=1)
y = votes_historcail_df_copy[['REP_VOTES%', 'YEAR']].values

In [41]:
X_train, X_test, y_train, y_test = train_test_split_by_year(X, y, year)

In [42]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [43]:
y_pred = linear_model.predict(X_test)

In [44]:
print("Accuracy score={:.2f}".format(linear_model.score(X_test, y_test) * 100))

Accuracy score=55.64


In [45]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.076685018440759
Mean Squared Error: 0.009344243792511753
Root Mean Squared Error: 0.09666562880627091


### Predict R and D Total votes per County###

In [46]:
votes_df_copy = votes_df[['STATE_FIPS', 'COUNTY_FIPS', 'COUNTY_TOTALVOTES', 'WINNING_PARTY_BINARY']]

In [47]:
y_pred = linear_model.predict(votes_df_copy)

In [48]:
target_year_data['REP_VOTES%'] = y_pred

In [49]:
#info D total votes. 
target_year_data['DEM_VOTES%'] = 1 - target_year_data['REP_VOTES%']

In [50]:
target_year_data['REP_VOTES'] = target_year_data['REP_VOTES%'] * target_year_data['COUNTY_TOTALVOTES']
target_year_data['DEM_VOTES'] = target_year_data['DEM_VOTES%'] * target_year_data['COUNTY_TOTALVOTES']

In [51]:
target_year_data['REP_VOTES'] = target_year_data['REP_VOTES'].astype(np.int64)
target_year_data['DEM_VOTES'] = target_year_data['DEM_VOTES'].astype(np.int64)

In [52]:
#save to csv
filename_step3 = '../../data/F20_step3_output_' + date_str + '.csv'
target_year_data.to_csv(filename_step3)

# Step 4

### Calculate Winner###

In [79]:
electoral_ref = pd.read_excel('../../data/Electoral College Votes.xlsx')
target_data = pd.read_csv(filename_step3)
data_historical = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")
states_aggr = pd.read_csv('../../data/state_aggregated_0726.csv')

In [80]:
states_aggr

,YEAR,STATE_FIPS,STATE,TOTAL_DEM_VOTES,TOTAL_REP_VOTES,TOTAL_VOTES,MARGIN_VICTORY,ABS_MARGIN_VICTORY,WINNING_PARTY,ELECTORAL_VOTES,TOP_TOPIC
0,2012,1,Alabama,"795,696","1,255,925","2,051,621",0.224325,0.224325,1.0,9,Economy
1,2012,2,Alaska,"142,458","192,750","335,208",0.150032,0.150032,1.0,3,Economy
2,2012,4,Arizona,"1,025,232","1,233,654","2,258,886",0.092268,0.092268,1.0,11,Economy
3,2012,5,Arkansas,"394,409","647,744","1,042,153",0.243088,0.243088,1.0,6,Economy
4,2012,6,California,"7,854,285","4,839,958","12,694,243",-0.237456,0.237456,0.0,55,Economy
...,...,...,...,...,...,...,...,...,...,...,...
308,2020,51,Virginia,NaN,NaN,NaN,NaN,NaN,NaN,13,Economy
309,2020,53,Washington,NaN,NaN,NaN,NaN,NaN,NaN,12,Economy
310,2020,54,West Virginia,NaN,NaN,NaN,NaN,NaN,NaN,5,Economy
311,2020,55,Wisconsin,NaN,NaN,NaN,NaN,NaN,NaN,10,Immigration


In [81]:
### 1) create a new df with StateFips/Count
state_fips_map = {}
for index, row in data_historical.iterrows():
    state = row['STATE']
    if (state_fips_map.get(state) == None):
        state_fips_map[state] = row['STATE_FIPS']
        
state_fips_df = pd.DataFrame(list(state_fips_map.items()), columns=['STATE','STATE_FIPS'])

In [82]:
df = electoral_ref.merge(state_fips_df, left_on='STATE', right_on='STATE')

In [83]:
target_data.drop('Unnamed: 0', inplace=True, axis=1)

In [84]:
"""extract how counties prediction for every state"""
visited = {}
electoral_votes = {'D':0, 'R':0 }
for index, row in target_data.iterrows():
    state = int(row['STATE_FIPS'])
    
    rep_votes = int(row['REP_VOTES'])
    dem_votes = int(row['DEM_VOTES'])

    if (visited.get(state) == None):
        electoral_votes = {'D': dem_votes, 'R': rep_votes }
        visited[state] = electoral_votes
    else:
        #update existing map
        current_votes = visited.get(state)
        current_r = current_votes.get('R')
        current_d = current_votes.get('D')
        total_dem = current_d + dem_votes
        total_rep = current_r + rep_votes
        electoral_votes = {'D': total_dem, 'R': total_rep }
        visited[state] = electoral_votes

In [85]:
votes_pred_df = pd.DataFrame(list(visited.items()), columns=['STATE_FIPS','PRED_VOTES'])

In [86]:
for index, row in votes_pred_df.iterrows():
    key_val = row['PRED_VOTES']
    d_votes = key_val.get('D')
    r_votes = key_val.get('R')
    votes_pred_df.loc[index, 'PRED_DEM_VOTES'] = key_val.get('D')
    votes_pred_df.loc[index, 'PRED_REP_VOTES'] = key_val.get('R')

In [87]:
votes_pred_df.drop('PRED_VOTES', inplace=True, axis=1)
votes_pred_df['PRED_DEM_VOTES'] = votes_pred_df['PRED_DEM_VOTES'].astype(np.int64)
votes_pred_df['PRED_REP_VOTES'] = votes_pred_df['PRED_REP_VOTES'].astype(np.int64)
votes_pred_df

,STATE_FIPS,PRED_DEM_VOTES,PRED_REP_VOTES
0,1,1037054,1083735
1,2,242744,223455
2,4,1779734,820395
3,5,504135,715696
4,6,10310902,3766648
5,8,1438440,1081870
6,9,1009105,628031
7,10,254355,191128
8,11,175106,99503
9,12,4778981,4164215


In [88]:
"""determine the winner of every state"""
for index, row in votes_pred_df.iterrows():
    
    total_r = row['PRED_REP_VOTES']
    total_d = row['PRED_DEM_VOTES']
    
    votes_pred_df.loc[index, 'PRED_WINNING_PARTY'] = '0' if total_d > total_r else '1'

In [89]:
votes_pred_df

,STATE_FIPS,PRED_DEM_VOTES,PRED_REP_VOTES,PRED_WINNING_PARTY
0,1,1037054,1083735,1
1,2,242744,223455,0
2,4,1779734,820395,0
3,5,504135,715696,1
4,6,10310902,3766648,0
5,8,1438440,1081870,0
6,9,1009105,628031,0
7,10,254355,191128,0
8,11,175106,99503,0
9,12,4778981,4164215,0


In [90]:
votes_pred_df

,STATE_FIPS,PRED_DEM_VOTES,PRED_REP_VOTES,PRED_WINNING_PARTY
0,1,1037054,1083735,1
1,2,242744,223455,0
2,4,1779734,820395,0
3,5,504135,715696,1
4,6,10310902,3766648,0
5,8,1438440,1081870,0
6,9,1009105,628031,0
7,10,254355,191128,0
8,11,175106,99503,0
9,12,4778981,4164215,0


In [91]:
data_aggr_2020 = states_aggr.loc[states_aggr['YEAR'] == 2020]
data_aggr_2016 = states_aggr.loc[states_aggr['YEAR'] == 2016]
data_aggr_2012 = states_aggr.loc[states_aggr['YEAR'] == 2012]


In [92]:
join_df = None
if year == 2012:
    join_df = data_aggr_2012
elif year == 2016:
    join_df = data_aggr_2016
else:
    join_df = data_aggr_2020

In [93]:
data_aggr_2020

,YEAR,STATE_FIPS,STATE,TOTAL_DEM_VOTES,TOTAL_REP_VOTES,TOTAL_VOTES,MARGIN_VICTORY,ABS_MARGIN_VICTORY,WINNING_PARTY,ELECTORAL_VOTES,TOP_TOPIC
255,2020,1,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,9,Economy
256,2020,2,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,3,Economy
257,2020,4,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,11,Economy
258,2020,5,Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,6,Economy
259,2020,6,California,NaN,NaN,NaN,NaN,NaN,NaN,55,Economy
260,2020,8,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,9,Climate Change
261,2020,9,Connecticut,NaN,NaN,NaN,NaN,NaN,NaN,7,Economy
262,2020,1,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,9,Economy
263,2020,2,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,3,Economy
264,2020,4,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,11,Economy


In [94]:
pred_actual_df = pd.merge(votes_pred_df, join_df, on='STATE_FIPS')

In [95]:
pred_actual_df

,STATE_FIPS,PRED_DEM_VOTES,PRED_REP_VOTES,PRED_WINNING_PARTY,YEAR,STATE,TOTAL_DEM_VOTES,TOTAL_REP_VOTES,TOTAL_VOTES,MARGIN_VICTORY,ABS_MARGIN_VICTORY,WINNING_PARTY,ELECTORAL_VOTES,TOP_TOPIC
0,1,1037054,1083735,1,2020,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,9,Economy
1,1,1037054,1083735,1,2020,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,9,Economy
2,2,242744,223455,0,2020,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,3,Economy
3,2,242744,223455,0,2020,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,3,Economy
4,4,1779734,820395,0,2020,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,11,Economy
5,4,1779734,820395,0,2020,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,11,Economy
6,5,504135,715696,1,2020,Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,6,Economy
7,5,504135,715696,1,2020,Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,6,Economy
8,6,10310902,3766648,0,2020,California,NaN,NaN,NaN,NaN,NaN,NaN,55,Economy
9,6,10310902,3766648,0,2020,California,NaN,NaN,NaN,NaN,NaN,NaN,55,Economy


In [96]:
pred_actual_df.drop('YEAR', inplace=True, axis=1)

In [97]:
pred_actual_df.columns

Index(['STATE_FIPS', 'PRED_DEM_VOTES', 'PRED_REP_VOTES', 'PRED_WINNING_PARTY',
       'STATE', 'TOTAL_DEM_VOTES', 'TOTAL_REP_VOTES', 'TOTAL_VOTES',
       'MARGIN_VICTORY', 'ABS_MARGIN_VICTORY', 'WINNING_PARTY',
       'ELECTORAL_VOTES', 'TOP_TOPIC'],
      dtype='object')

In [98]:
pred_actual_df.head()

,STATE_FIPS,PRED_DEM_VOTES,PRED_REP_VOTES,PRED_WINNING_PARTY,STATE,TOTAL_DEM_VOTES,TOTAL_REP_VOTES,TOTAL_VOTES,MARGIN_VICTORY,ABS_MARGIN_VICTORY,WINNING_PARTY,ELECTORAL_VOTES,TOP_TOPIC
0,1,1037054,1083735,1,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,9,Economy
1,1,1037054,1083735,1,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,9,Economy
2,2,242744,223455,0,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,3,Economy
3,2,242744,223455,0,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,3,Economy
4,4,1779734,820395,0,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,11,Economy


In [99]:
file_name = 'actuals_predictions_state_' + str(year) + '.csv'
pred_actual_df.to_csv(file_name)